# Introduction to Pandas

Pandas is a library for reading data frames in Python. It can be considered an "advanced" version of the standard `csv` module, with more powerful data table maniputations and SQL-like operations. Similar to Numpy, Pandas is highly optimized and can speed up our code considerably if used correctly.

We start by importing the `pandas` module.

In [1]:
import pandas as pd

## 1. Creating Data frame

Some common data structures that can be read by Pandas include:

1) Tabular text file with delimiter (e.g., csv, tsv)

In [2]:
filepath = "https://raw.githubusercontent.com/wireservice/csvkit/master/examples/foo1.csv"
df = pd.read_csv(filepath)
df

,id,name,age
0,1,Jake,22
1,2,Howard,21


2) 2D array-like structure

In [3]:
df = pd.DataFrame([[1, 2, 3], [4, 5, 6]], columns = ['col1', 'col2', 'col3'])
df

,col1,col2,col3
0,1,2,3
1,4,5,6


3) Dictionary that maps column name (string) to column values (array-like)

In [4]:
df = pd.DataFrame({"ID" : [1, 2, 3], "First Name" : ["John", "Jim", "Joe"], "Last Name" : ["Smith", "Hendry", "Wilson"]})
df

,ID,First Name,Last Name
0,1,John,Smith
1,2,Jim,Hendry
2,3,Joe,Wilson


For more details, refer to the `pd.DataFrame` [constructor](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

## 2. Accessing data frame
### 2.1 Basic access operations

Accessing a particular cell in a data frame is done with either `.loc` or `.iloc`, followed by a squared bracket, in the same manner as a Python list or Numpy array. Slicing by rows or columns is also supported with the syntax `start_index:end_index`.

In [5]:
print('Get first 2 rows of data frame')
display(df.head(2))

# index into a dataframe with df.loc[rows, columns] and df.iloc[row numbers, column numbers]
print('Series of all last names')
display(df.loc[:, "Last Name"])
display(df["Last Name"]) # alternatively, can omit the row indexing to get the entire column

print('DataFrame with one column "Last Name"')
display(df.loc[:, ["Last Name"]])

print('DataFrame with only rows at index 1, 2')
display(df.loc[[1,2], :])

print('Set an entry in a DataFrame')
df.loc[1,"Last Name"] = "Kilter" 
display(df)

print('# Set row at index 3. If input index does not exist, new row is appeneded to the end')
df.loc[3,:] = (100, "Andrew", "Moore") 
display(df)

print('Get cell at row 0 and column 0')
display(df.iloc[0,0])

Get first 2 rows of data frame


,ID,First Name,Last Name
0,1,John,Smith
1,2,Jim,Hendry


Series of all last names


0     Smith
1    Hendry
2    Wilson
Name: Last Name, dtype: object

0     Smith
1    Hendry
2    Wilson
Name: Last Name, dtype: object

...

,ID,First Name,Last Name
0,1,John,Smith
1,2,Jim,Kilter
2,3,Joe,Wilson


# Set row at index 3. If input index does not exist, new row is appeneded to the end


,ID,First Name,Last Name
0,1.0,John,Smith
1,2.0,Jim,Kilter
2,3.0,Joe,Wilson
3,100.0,Andrew,Moore


Get cell at row 0 and column 0


1.0

### 2.2 Querying data frame

Typically we want to select a subset of the rows that satisfy some conditions. Pandas allows us to do in a way similar to boolean array indexing in Numpy:

In [6]:
display(df)

# find rows where First Name is Jim and Last Name is Kilter
df[(df["First Name"] == "Jim") & (df["Last Name"] == "Kilter")]

,ID,First Name,Last Name
0,1.0,John,Smith
1,2.0,Jim,Kilter
2,3.0,Joe,Wilson
3,100.0,Andrew,Moore


,ID,First Name,Last Name
1,2.0,Jim,Kilter


As Pandas supports more than just numerical data types, there are many built-in methods to help with constructing the boolean indexing:

In [7]:
# find rows where Last Name has 6 characters
display(df[df["Last Name"].str.len() == 6])

# find rows where First Name contains the substring "Jo"
display(df[df["First Name"].str.contains("Jo")])

# find rows where First Name is either "Jim" or "Kim"
display(df[df["First Name"].isin(["Jim", "Kim"])])

,ID,First Name,Last Name
1,2.0,Jim,Kilter
2,3.0,Joe,Wilson


,ID,First Name,Last Name
0,1.0,John,Smith
2,3.0,Joe,Wilson


,ID,First Name,Last Name
1,2.0,Jim,Kilter


Note that the first two methods are specific to the string data type, so they require a conversion to string via `.str`.

If we instead want to find rows that do not satisfy a certain condition, the negation symbol `~` can be used:

In [8]:
# find rows where First Name is neither "Jim" nor "Kim"
display(df[~df["First Name"].isin(["Jim", "Kim"])])

,ID,First Name,Last Name
0,1.0,John,Smith
2,3.0,Joe,Wilson
3,100.0,Andrew,Moore


Pandas also support using a query string to select rows, which can avoid the creation of the intermediate boolean index and reduce runtime / memory usage:

In [9]:
# find rows where First Name is John and Last Name is Smith
display(df.query('(`First Name` == "John") & (`Last Name` == "Smith")'))

# find rows where First Name is either "Jim" or "Kim"
display(df.query('`First Name` in ["Jim", "Kim"]'))

,ID,First Name,Last Name
0,1.0,John,Smith


,ID,First Name,Last Name
1,2.0,Jim,Kilter


For more information, refer to the [.query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) and this [tutorial](https://cmdlinetips.com/2019/07/how-to-select-rows-of-pandas-dataframe-with-query-function/).

There are two important points to note about querying:
1) The returned object of a query is a view of the original data frame. Modifying the view will not affect the original data frame, but will yield a warning:

In [10]:
df_view = df[df["Last Name"].str.len() == 6]
df_view["First Name"] = ["Kim", "Koe"]

print("Modified view")
display(df_view)

print("Original data frame")
display(df)

Modified view


/Users/hn1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,First Name,Last Name
1,2.0,Kim,Kilter
2,3.0,Koe,Wilson


Original data frame


,ID,First Name,Last Name
0,1.0,John,Smith
1,2.0,Jim,Kilter
2,3.0,Joe,Wilson
3,100.0,Andrew,Moore


To avoid this warning, we can call `.copy` on the view to turn it into an independent data frame:

In [11]:
df_copy = df[df["Last Name"].str.len() == 6].copy()
df_copy["First Name"] = ["Kim", "Koe"]
df_copy

,ID,First Name,Last Name
1,2.0,Kim,Kilter
2,3.0,Koe,Wilson


2) Note that, unlike Numpy, Pandas preserves the original row index after filtering. For example, in `df_copy` above, to access the first row we would need to call `df_copy.loc[1,:]`. Calling `df_copy.loc[0,:]` will throw a `KeyError` exception:

In [12]:
print("This works:")
display(df_copy.loc[1,:])

print("This doesn't work:")
df_copy.loc[0,:]

This works:


ID                 2
First Name       Kim
Last Name     Kilter
Name: 1, dtype: object

This doesn't work:


KeyError: 0

To address this issue, we can call `.reset_index` so that the first row has index 0, the second row as index 1, and so on, as usual.

In [13]:
df_copy_reset_index = df_copy.reset_index(drop = True)

display(df_copy_reset_index)

df_copy_reset_index.loc[0, "First Name"]

,ID,First Name,Last Name
0,2.0,Kim,Kilter
1,3.0,Koe,Wilson


'Kim'

### 2.2 Iterating over data frame

There are two common ways to iterate over the columns of a data frame:

1. Call `.columns` to get the list of column names and iterate over it
2. Use `.iloc` along with the column indexes

In [14]:
for col in df.columns:
    # do something with df[col], which is a Series
    df[col]
    
for col_index in range(df.shape[1]):
    # do something with df[:,col_index], which is a Series
    df.iloc[:, col_index]

There are four common ways to iterate over the rows of a data frame:
1. Use `.iloc` along with row index.
2. Use `iterrows` method.
3. Use `apply` with `axis=1`.
4. Use Pandas vectorization

Often while doing row iteration we want to map each row to an output value, and the resulting collection of output values can be used for other purposes, e.g., becoming a new column.
As an example, let's perform row iteration on our data frame to create a new column called Full Name, using each of the 4 ways above:

In [15]:
def full_name(first, last):
    return first + " " + last

# Use .iloc along with row index
for i in range(len(df)):
    df.loc[i, "Full Name 1"] = full_name(df.loc[i, "First Name"], df.loc[i, "Last Name"])

# Use iterrows method. Note the row returned by .iterrows() is only a copy of the data,
# so we cannot update the data frame during .iterrows()
full_names = []
for row_index, row in df.iterrows():
    full_names.append(full_name(row["First Name"], row["Last Name"]))
df["Full Name 2"] = full_names

# Use apply with axis=1. By default, axis=0 will loop through the columns.
df["Full Name 3"] = df.apply(lambda row: full_name(row["First Name"], row["Last Name"]), axis=1)

# Use Pandas vectorization
df["Full Name 4"] = full_name(df["First Name"], df["Last Name"])

df

,ID,First Name,Last Name,Full Name 1,Full Name 2,Full Name 3,Full Name 4
0,1.0,John,Smith,John Smith,John Smith,John Smith,John Smith
1,2.0,Jim,Kilter,Jim Kilter,Jim Kilter,Jim Kilter,Jim Kilter
2,3.0,Joe,Wilson,Joe Wilson,Joe Wilson,Joe Wilson,Joe Wilson
3,100.0,Andrew,Moore,Andrew Moore,Andrew Moore,Andrew Moore,Andrew Moore


The question, then, is which iteration technique should we use. In a nutshell, **the four techniques above are ordered from slowest to fastest**, i.e., using `.iloc` is slowest and Pandas vectorization is fastest.

Here Pandas vectorization refers to the process of operating a procedure on the *entire column array at once*, instead of on individual column elements. This can be achieved in two ways:

1) Use built-in `pd.Series` methods. For example, to convert a String column to lowercase through vectorization, we can simply call `.lower`:

In [16]:
df["First Name"].str.lower()

0      john
1       jim
2       joe
3    andrew
Name: First Name, dtype: object

2) Use operations that are compatible with Numpy arrays, for example basic math operations or Boolean conditons. For example, in the two implementations of `full_name` below, the first can be vectorized because it uses `+`, while the second cannot, even though both peform the same task.

In [17]:
def full_name_vectorized(first, last):
    return first + " " + last

def full_name_not_vectorized(first, last):
    return f"{first} {last}"

Naturally, not all operations can be vectorized, so an acceptable middle ground is using `apply`, which can work with any input function. In case vectorization is possible and the involved columns are numerical, we can also apply vectorization on the underlying Numpy arrays (by calling `df[column_name].to_numpy()`) for an even greater speedup.

To see how much speedup is gained by vectorization (short answer: a lot!), refer to the following guides:
* [A Beginner's Guide to Optimizing Pandas Code for Speed](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6)
* [Vectorization and Parallelization in Python with Numpy and Pandas](https://datascience.blog.wzb.eu/2018/02/02/vectorization-and-parallelization-in-python-with-numpy-and-pandas/)

The high-level lesson here is to avoid loops whenever possible, just like when working with Numpy. The best way to loop is not looping at all :)

## 3. Manipulating data frame
One primary advantage of Pandas over other data table packages is its powerful data manipulation functions. Here we introduce some common themes.

Before we start, note that *most* dataframe methods do not modify the input dataframe and only return the output in a new dataframe. If you want to modify a dataframe in-place, either look for the parameter `inplace` in the method API, or reassign your dataframe to the method output. For example, to replace every missing entry in a dataframe `df` with 0, you can either use `df.fillna(0, inplace = True)` or `df = df.fillna(0)`.


### 3.1 Conversion between long and wide formats

Input data often comes in two forms: long and wide. In the wide format, every row represents a unique observation and every column represents a feature. For example, in the following data, the rows are distinct countries and the columns are relevant attributes of each country:

In [18]:
df_wide = pd.DataFrame({
    "country" : ["A", "B", "C"],
    "population_in_million" : [100, 200, 120],
    "gdp_percapita" : [2000, 7000, 15000]
})
df_wide

,country,population_in_million,gdp_percapita
0,A,100,2000
1,B,200,7000
2,C,120,15000


In the long format, there is one column for the observation ID, one column for attribute name, and one for attribute value. This is often easier to implement, as addition of a new feature does not change the table structure; however, it's harder to understand.

In [19]:
df_long = pd.DataFrame({
    "country" : ["A", "A", "B", "B", "C", "C"],
    "attribute" : ["population_in_million", "gdp_percapita"] * 3,
    "value" : [100, 2000, 200, 7000, 120, 15000]
})
df_long

,country,attribute,value
0,A,population_in_million,100
1,A,gdp_percapita,2000
2,B,population_in_million,200
3,B,gdp_percapita,7000
4,C,population_in_million,120
5,C,gdp_percapita,15000


The long format is useful when you are curating data and do not yet know what the final structure will be. When your data is ready for analysis, the wide format is preferred.

To convert from wide to long, we use `.melt` with four parameters:
1. `id_vars`: names of the columns with the observation IDs
2. `value_vars`: names of the feature columns
3. `var_name`: name of the new column that will contain the feature names
3. `value_name`: name of the new column that will contain the feature values.

In [20]:
df_wide.melt(id_vars = ["country"], value_vars = ["population_in_million", "gdp_percapita"], var_name = "attribute", value_name = "value")

,country,attribute,value
0,A,population_in_million,100
1,B,population_in_million,200
2,C,population_in_million,120
3,A,gdp_percapita,2000
4,B,gdp_percapita,7000
5,C,gdp_percapita,15000


To convert from long to wide, we use `.pivot_table` with three parameters:
1. `index`: name of the column with the ids.
2. `columns`: name of the column that contains the feature names.
3. `values`: name of the column that contains the feature values.

In [21]:
df_long.pivot_table(index = "country", columns = "attribute", values = "value")

attribute,gdp_percapita,population_in_million
country,,
A,2000,100
B,7000,200
C,15000,120


### 3.2 Groupby: split-apply-combine

`groupby` is an SQL-like operation that partitions your data frame into groups based on the values at some columns.
As an example, consider the following dataset:

In [22]:
df = pd.DataFrame({
    "city" : ["Pittsburgh", "San Francisco", "Los Angeles", "Seattle", "Philadelphia"],
    "state" : ["PA", "CA", "CA", "WA", "PA"],
    "population" : [301048, 883305, 3990456, 744955, 1584138],
    "area (miles)" : [58.34, 46.89, 503, 83.78, 141.7]
})
df

,city,state,population,area (miles)
0,Pittsburgh,PA,301048,58.34
1,San Francisco,CA,883305,46.89
2,Los Angeles,CA,3990456,503.00
3,Seattle,WA,744955,83.78
4,Philadelphia,PA,1584138,141.70


Let's say we want to see how many cities are present in each state. This is done by grouping the dataset by state and count the number of rows in each group (assuming each city occupies only one row):

In [23]:
df.groupby("state").count()

,city,population,area (miles)
state,,,
CA,2,2,2
PA,2,2,2
WA,1,1,1


`.count()` is called an aggregation function, which takes as input all the rows in a group and outputs one value (in this case the number of rows). There are many other aggregation functions such as `.max(), .min(), .sum()`.

To apply multiple aggregation functions to each group, we can call `.agg`, which takes a mapping from column name to aggregation functions:

In [24]:
df.groupby("state").agg({"city" : "count", "population" : ["sum", "max"]})

city population         
      count        sum      max
state                          
CA        2    4873761  3990456
PA        2    1885186  1584138
WA        1     744955   744955

More generally, an important technique in Pandas groupby is the [split-apply-combine pattern](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html):
* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure

Splitting is done via `.groupby()` itself. Applying is done by calling `apply()` along with the specified input functions (which can be aggregation, transformation, filtration or a combination of them). Finally, combining is done automatically on the returned values of `apply`.

As an example, for the above dataset, let's say we want to compute the total population and population density (population / area) of the states that have at least two cities present in the dataset.

In [25]:
df

,city,state,population,area (miles)
0,Pittsburgh,PA,301048,58.34
1,San Francisco,CA,883305,46.89
2,Los Angeles,CA,3990456,503.00
3,Seattle,WA,744955,83.78
4,Philadelphia,PA,1584138,141.70


In [26]:
def process_group(group):
    # group is a DataFrame where all the rows belong to one common state
    output = {}
    # only consider states with at least two cities
    if len(group) > 1:
        output["total population"] = group["population"].sum()
        output["population density"] = output["total population"] / group["area (miles)"].sum()
    # Series indexes become new column labels
    return pd.Series(output, index = ["total population", "population density"], dtype = "float64")

result = df.groupby("state").apply(process_group)
result

,total population,population density
state,,
CA,4873761.0,8863.156268
PA,1885186.0,9424.045191
WA,NaN,NaN


The state `WA` does not satisfy the condition of having at least two cities, so it gets a row with only NAN values. We can remove it with a call to `dropna()`:

In [27]:
result.dropna()

,total population,population density
state,,
CA,4873761.0,8863.156268
PA,1885186.0,9424.045191


Note that earlier we also used `.agg()` to compute the total population by state. However, `.agg()` can only aggregate data in each column separately, so it cannot compute the population density, which involves two columns. `apply` is a more general version of `agg` that can handle multi-column operations while also performing filtration, as we did with the `WA` state.

**Important note**: Make sure to check that the columns which you perform groupby on have no empty values (`np.nan`). These will be ignored during groupby, resulting in potential loss of data.

## 4. Working with multiple data frames

Pandas offer several functions for handling multiple data frames. Here we outline some common functions.

### 4.1 Concatenation

Data frames can be concatenated along the rows or columns using `pd.concat`.

In [28]:
df1 = pd.DataFrame({
    "col1" : [1, 2, 3],
    "col2" : [4, 5, 6]
})

df2 = pd.DataFrame({
    "col1" : [7, 8],
    "col2" : [9, 10]
})

df3 = pd.DataFrame({
    "col3" : [11, 12],
    "col4" : [13, 14]
})

print("concatenate along rows")
display(pd.concat([df1, df2]))

print("concatenate along columns")
display(pd.concat([df1, df3], axis = 1))

concatenate along rows


,col1,col2
0,1,4
1,2,5
2,3,6
0,7,9
1,8,10


concatenate along columns


,col1,col2,col3,col4
0,1,4,11.0,13.0
1,2,5,12.0,14.0
2,3,6,NaN,NaN


### 4.2 Joining

Pandas supporting joining data frames in an SQL style, where we can specify the columns / indexes to join as well as the joining type:
* Left join keeps all rows of the left table, add entries from right table that match the corresponding columns.
* Right join is the like a left join but with the roles of the tables reversed.
* Outer join returns all rows from both left and right join.
* Inner join return the rows where the two joined columns
contain the same value

Joining is done via the `merge` method: 

In [29]:
df1 = pd.DataFrame({
    "col1" : [1, 2, 3],
    "col2" : [4, 5, 6]
})

df2 = pd.DataFrame({
    "col1" : [1, 2, 4],
    "col_3" : [9, 10, 11]
})

print("left join")
display(df1.merge(df2, left_on = "col1", right_on = "col1", how = "left"))

print("right join")
display(df1.merge(df2, left_on = "col1", right_on = "col1", how = "right"))

print("inner join")
display(df1.merge(df2, left_on = "col1", right_on = "col1", how = "inner"))

print("outer join")
display(df1.merge(df2, left_on = "col1", right_on = "col1", how = "outer"))

left join


,col1,col2,col_3
0,1,4,9.0
1,2,5,10.0
2,3,6,NaN


right join


,col1,col2,col_3
0,1,4.0,9
1,2,5.0,10
2,4,NaN,11


inner join


,col1,col2,col_3
0,1,4,9
1,2,5,10


outer join


,col1,col2,col_3
0,1,4.0,9.0
1,2,5.0,10.0
2,3,6.0,NaN
3,4,NaN,11.0


Note that if the column to merge on is an index, we need to use `left_index = True` instead of `left_on = <column name>` (same for `right_index` vs `right_on`).

In [30]:
df1.set_index("col1", inplace = True)
df1.merge(df2, left_index = True, right_on = "col1", how = "left")

,col2,col1,col_3
0.0,4,1,9.0
1.0,5,2,10.0
NaN,6,3,NaN


There is also a `.join` method that works similarly to `.merge` but only assumes joining by indexes, not by column names. Joining by indexes is actually faster, so if your data frames are indexed, consider using `.join` instead.